## Universidad Central
### Maestría en Analítica de datos
### Mabel Ayala Meneses
### Proyecto final Big DAta
### Web Scrapping

# ==========================
# 1. IMPORTS Y CONFIGURACIÓN
# ==========================

In [ ]:
!pip install pandas openpyxl requests

In [ ]:
import os
import pandas as pd
import requests
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Tiempo de espera en la descarga (conexión, lectura)
REQUEST_TIMEOUT = (10, 40)

# Carpeta donde se guardarán los PDFs descargados
CARPETA_PDFS = "/content/drive/MyDrive/Big Data/PDFs"
os.makedirs(CARPETA_PDFS, exist_ok=True)

In [ ]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
REQUEST_TIMEOUT = (10, 40)   # (conexión, lectura)
MAX_PDF_MB = 25              # tamaño máximo de PDF en MB

# ==========================
# 3. DEFINIR COLUMNAS A USAR
# ==========================

In [ ]:
RUTA_EXCEL = "/content/Lenguaje controlado_Sistemas Informacion_10102025.xlsx"
df = pd.read_excel(RUTA_EXCEL, sheet_name="Diccionario")

# Renombramos columnas para que sean más manejables
df = df.rename(columns={
    # "Servicio SDMujer": "servicio_sdmujer",
    "definicion_termino": "definicion_1",
    "fuente_termino": "fuente_1",
    "link_fuente_termino_": "link_1",
    "definicion_termino2": "definicion_2",
    "fuente_termino_2": "fuente_2",
    "link_fuente_termino_2": "link_2",
    "definicion_termino3": "definicion_3",
    "fuente_termino_3": "fuente_3",
    "link_fuente_termino_3": "link_3",
    "termino_padre": "termino_padre",
    "termino_hijo": "termino_hijo",
    "subtermino": "subtermino",
    "subtermino1": "subtermino1",
    "Observaciones": "observaciones",
    "sistema_informacion": "sistema_informacion",
    "modulo": "modulo",
    "submodulo": "submodulo",
    "tema": "tema",
    "subtema": "subtema"
})

columnas_interes = [
    #"cod",
    #"sistema_informacion",
    #"modulo",
    #"submodulo",
    #"tema",
    #"subtema",
    "termino_padre",
    "termino_hijo",
    #"subtermino",
    #"subtermino1",
    #"servicio_sdmujer",
    "definicion_1", "fuente_1", "link_1",
    "definicion_2", "fuente_2", "link_2",
    "definicion_3", "fuente_3", "link_3",
    "terminos_relacionados",
    #"observaciones"
]

df_base = df[columnas_interes].copy()
df_base.head()



,termino_padre,termino_hijo,definicion_1,fuente_1,link_1,definicion_2,fuente_2,link_2,definicion_3,fuente_3,link_3,terminos_relacionados
0,Acceso a la Justicia,NaN,El derecho de acceso a la administración de ju...,Sentencia T-608/19. Corte Constitucional Repúb...,https://www.cidh.oas.org/women/acceso07/cap1.h...,NaN,NaN,NaN,NaN,NaN,NaN,"Garantía judicial, Derecho a la defensa, Admin..."
1,Activación de rutas,NaN,Acciones que efectúa la profesional psicosocia...,GUÍA DE ATENCIÓN EQUIPO TRANSVERSAL PSICOSOCIA...,https://kawak.com.co/sdmujer/gst_documental/do...,"Hace referencia a la remisión, por los diferen...","GUÍA PARA LA ATENCIÓN DE NIÑAS, NIÑOS Y ADOLES...",https://kawak.com.co/sdmujer/gst_documental/do...,Acciones que tienen como objetivo tramitar art...,Manual del modelo de Atención Rural Casas de i...,https://kawak.com.co/sdmujer/mapa_procesos/map...,"Derivación de casos, canalización, referencia,..."
2,Activación de rutas,Activación de ruta de riesgos de feminicidio,acción encaminada a prevenir la materializació...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Activación de rutas,Apoyo especializado ataque con agente químico,Activación de rutas de atención y protección p...,GUÍA PARA EL REDIRECCIONAMIENTO DE CASOS DE MU...,https://kawak.com.co/sdmujer/gst_documental/do...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Activación de rutas,Apoyo especializado trata de personas,Activación de rutas de atención y protección p...,GUÍA PARA EL REDIRECCIONAMIENTO DE CASOS DE MU...,https://kawak.com.co/sdmujer/gst_documental/do...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# ==========================
# 4. CONSTRUIR TABLA SOLO CON PDFs
# ==========================

In [ ]:
## Esta es la ultima versión
def extraer_urls_pdf_de_celda(valor):
    """
    Recibe el contenido de una celda (puede tener 0, 1 o muchas URLs).
    Devuelve una lista de URLs que apunten a PDFs.

    - Acepta URLs que terminen en .pdf (antes de ? o #)
    - Acepta URLs de Función Pública que usan norma.php o norma_pdf.php
      porque aunque no terminen en .pdf, sirven un PDF.
    """
    if pd.isna(valor):
        return []

    texto = str(valor).replace("\r", " ").replace("\n", " ")
    tokens = texto.split()

    urls_pdf = []

    for t in tokens:
        t = t.strip().strip('",\'')  # quita comillas y comas alrededor

        # Solo nos interesan http / https
        if not (t.startswith("http://") or t.startswith("https://")):
            continue

        # Quitamos parámetros (?utm_source...) y anchors (#algo)
        base = t.split("?")[0].split("#")[0]
        base_lower = base.lower()

        es_pdf_por_extension = base_lower.endswith(".pdf")

        # Caso especial: gestornormativo de Función Pública
        es_funcion_publica_norma = (
            "funcionpublica.gov.co" in base_lower
            and "/eva/gestornormativo/" in base_lower
            and ("norma.php" in base_lower or "norma_pdf.php" in base_lower)
        )

        if es_pdf_por_extension or es_funcion_publica_norma:
            urls_pdf.append(t)

    return urls_pdf



In [ ]:
filas_pdf = []

for idx, row in df_base.iterrows():
    for origen_link in ["link_1", "link_2", "link_3"]:
        urls = extraer_urls_pdf_de_celda(row.get(origen_link))

        for url in urls:
            filas_pdf.append({
                "termino_padre": row["termino_padre"],
                "termino_hijo": row["termino_hijo"],
                "definicion_1": row["definicion_1"],
                "fuente_1": row["fuente_1"],
                "definicion_2": row["definicion_2"],
                "fuente_2": row["fuente_2"],
                "definicion_3": row["definicion_3"],
                "fuente_3": row["fuente_3"],
                "terminos_relacionados": row["terminos_relacionados"],
                "origen_link": origen_link,
                "url_pdf": url
            })

df_pdfs = pd.DataFrame(filas_pdf)

print("Total filas con 'pdf':", len(df_pdfs))
df_pdfs.head()


Total filas con 'pdf': 170


,termino_padre,termino_hijo,definicion_1,fuente_1,definicion_2,fuente_2,definicion_3,fuente_3,terminos_relacionados,origen_link,url_pdf
0,Actividad Principal,NaN,Actividad que más contribuye al valor agregado...,Sistema de consulta de conceptos estandarizado...,NaN,Clasificación Industrial Internacional Uniform...,NaN,NaN,"Ocupación, empleo, trabajo habitual, Oficio, f...",link_2,https://www.dane.gov.co/files/sen/nomenclatura...
1,Actividad Principal,Estudiante,La educación es un derecho de la persona y un ...,"Constitución política de Colombia, 1991.\nArtí...",NaN,NaN,NaN,NaN,NaN,link_1,https://www.funcionpublica.gov.co/eva/gestorno...
2,Actividad Principal,Independiente,Persona natural que realiza su actividad econó...,Decreto No. 1601 de 2022.pdf\nTítulo 7. Artícu...,NaN,NaN,NaN,NaN,NaN,link_1,https://www.minsalud.gov.co/Normatividad_Nuevo...
3,Actividad Principal,Informal,Se refiere al tipo de actividad que carece de ...,SNA2008Spanish.pdf\nHoja 612 - Pag 561 / Punto...,NaN,NaN,NaN,NaN,NaN,link_1,https://unstats.un.org/Unsd/nationalaccount/do...
4,Actividad Principal,Incapacidad permanente,Se considera como incapacitado permanente parc...,Ley_776_de_2002.pdf\nPag. 2 - Artículo 5,NaN,NaN,NaN,NaN,NaN,link_1,https://www.funcionpublica.gov.co/eva/gestorno...


In [ ]:
# 1. Primeras apariciones (una fila por URL)
df_pdfs_sorted = df_pdfs.sort_values(["url_pdf", "termino_padre", "termino_hijo"]).copy()
df_pdfs_sorted["orden_en_url"] = df_pdfs_sorted.groupby("url_pdf").cumcount()

df_base_1 = df_pdfs_sorted[df_pdfs_sorted["orden_en_url"] == 0].copy()
print("Filas con orden_en_url = 0:", len(df_base_1))

# 2. Segundas apariciones (si existen)
df_segundas = df_pdfs_sorted[df_pdfs_sorted["orden_en_url"] == 1].copy()

faltantes = max(0, 170 - len(df_base_1))
print("Filas faltantes para llegar a 120:", faltantes)

df_segundas_sel = df_segundas.head(faltantes)
df_pdfs_filtrado = pd.concat([df_base_1, df_segundas_sel], ignore_index=False)

print("Total después de segundas apariciones:", len(df_pdfs_filtrado))

# 3. Si todavía faltan filas, tomamos más del resto (orden_en_url >= 2)
restantes = max(0, 170 - len(df_pdfs_filtrado))
print("Filas que aún faltan:", restantes)

if restantes > 0:
    # Filas que aún NO hemos usado
    usados_idx = df_pdfs_filtrado.index
    df_restantes = df_pdfs_sorted[~df_pdfs_sorted.index.isin(usados_idx)].copy()

    df_extra = df_restantes.head(restantes)
    df_pdfs_filtrado = pd.concat([df_pdfs_filtrado, df_extra], ignore_index=False)

print("Total filas seleccionadas:", len(df_pdfs_filtrado))

# Si quieres resetear índices al final:
df_pdfs_filtrado = df_pdfs_filtrado.reset_index(drop=True)



Filas con orden_en_url = 0: 81
Filas faltantes para llegar a 120: 89
Total después de segundas apariciones: 96
Filas que aún faltan: 74
Total filas seleccionadas: 170


In [ ]:
df_pdfs

,termino_padre,termino_hijo,definicion_1,fuente_1,definicion_2,fuente_2,definicion_3,fuente_3,terminos_relacionados,origen_link,url_pdf
0,Actividad Principal,NaN,Actividad que más contribuye al valor agregado...,Sistema de consulta de conceptos estandarizado...,NaN,Clasificación Industrial Internacional Uniform...,NaN,NaN,"Ocupación, empleo, trabajo habitual, Oficio, f...",link_2,https://www.dane.gov.co/files/sen/nomenclatura...
1,Actividad Principal,Estudiante,La educación es un derecho de la persona y un ...,"Constitución política de Colombia, 1991.\nArtí...",NaN,NaN,NaN,NaN,NaN,link_1,https://www.funcionpublica.gov.co/eva/gestorno...
2,Actividad Principal,Independiente,Persona natural que realiza su actividad econó...,Decreto No. 1601 de 2022.pdf\nTítulo 7. Artícu...,NaN,NaN,NaN,NaN,NaN,link_1,https://www.minsalud.gov.co/Normatividad_Nuevo...
3,Actividad Principal,Informal,Se refiere al tipo de actividad que carece de ...,SNA2008Spanish.pdf\nHoja 612 - Pag 561 / Punto...,NaN,NaN,NaN,NaN,NaN,link_1,https://unstats.un.org/Unsd/nationalaccount/do...
4,Actividad Principal,Incapacidad permanente,Se considera como incapacitado permanente parc...,Ley_776_de_2002.pdf\nPag. 2 - Artículo 5,NaN,NaN,NaN,NaN,NaN,link_1,https://www.funcionpublica.gov.co/eva/gestorno...
...,...,...,...,...,...,...,...,...,...,...,...
165,Tipo de formación,Semilleros de Empoderamiento,Son procesos formativos implementados por la S...,Secretaría Distrital de la Mujer,NaN,NaN,NaN,NaN,NaN,link_1,https://www.sdmujer.gov.co/sites/default/files...
166,Tipo de formación,Presentación Prueba Saber 11,La Prueba Saber 11° es un examen estandarizado...,Ministerio de Educación,NaN,NaN,NaN,NaN,NaN,link_1,https://www.mineducacion.gov.co/1759/articles-...
167,Tipo de formación,Feria de Educación Superior,Este evento semestral tiene como objetivo prin...,Secretaría Distrital de la Mujer,NaN,NaN,NaN,NaN,NaN,link_1,https://www.sdmujer.gov.co/sites/default/files...
168,Tipos de Violencia,NaN,"Cualquier acción u omisión, que le cause muert...",LEY 1257 DE 2008. Art 02,Tipos de violencia contra las mujeres y las niñas,https://www.unwomen.org/es/articulos/preguntas...,NaN,https://kawak.com.co/sdmujer/gst_documental/do...,"Violencia psicológica, física, económica, sexual.",link_1,https://www.funcionpublica.gov.co/eva/gestorno...


In [ ]:
def descargar_pdf(url, id_local):
    """
    Intenta descargar un PDF.
    Devuelve: (ruta_local_o_None, estado, detalle)
    """
    try:
        resp = requests.get(url, timeout=REQUEST_TIMEOUT, verify=False)
        resp.raise_for_status()

        content_type = resp.headers.get("Content-Type", "").lower()

        # Si el servidor no dice 'pdf' pero la URL termina en .pdf, igual lo aceptamos
        es_pdf_por_header = "pdf" in content_type
        es_pdf_por_url = url.split("?")[0].split("#")[0].lower().endswith(".pdf")

        if not (es_pdf_por_header or es_pdf_por_url):
            return None, "no_es_pdf", f"Content-Type: {content_type}"

        # Guardar archivo
        nombre_archivo = f"doc_{id_local:04d}.pdf"
        ruta_local = os.path.join(CARPETA_PDFS, nombre_archivo)

        with open(ruta_local, "wb") as f:
            f.write(resp.content)

        size_mb = len(resp.content) / (1024 * 1024)
        return ruta_local, "ok", f"{size_mb:.2f} MB"

    except Exception as e:
        return None, "error_descarga", str(e)


In [ ]:
df_pdfs = df_pdfs #_filtrado
rutas_locales = []
estados = []
detalles = []

for i, row in df_pdfs.iterrows():
    url = row["url_pdf"]
    print(f"[{i}] Descargando: {url}")
    ruta, estado, detalle = descargar_pdf(url, i)
    rutas_locales.append(ruta)
    estados.append(estado)
    detalles.append(detalle)

df_pdfs["ruta_pdf_local"] = rutas_locales
df_pdfs["estado_descarga"] = estados
df_pdfs["detalle_descarga"] = detalles

df_pdfs.head()


[0] Descargando: https://www.dane.gov.co/files/sen/nomenclatura/ciiu/CIIU_Rev_4_AC2020.pdf
[1] Descargando: https://www.funcionpublica.gov.co/eva/gestornormativo/norma.php?i=4125
[2] Descargando: https://www.minsalud.gov.co/Normatividad_Nuevo/Decreto%20No.%201601%20de%202022.pdf
[3] Descargando: https://unstats.un.org/Unsd/nationalaccount/docs/SNA2008Spanish.pdf
[4] Descargando: https://www.funcionpublica.gov.co/eva/gestornormativo/norma.php?i=16752
[5] Descargando: https://www.funcionpublica.gov.co/eva/gestornormativo/norma_pdf.php?i=40764
[6] Descargando: https://rtc-cea.cepal.org/sites/default/files/2021-09/Guia%20metodologica%20uso%20del%20tiempo.pdf
[7] Descargando: https://rtc-cea.cepal.org/sites/default/files/2021-09/Guia%20metodologica%20uso%20del%20tiempo.pdf
[8] Descargando: https://rtc-cea.cepal.org/sites/default/files/2021-09/Guia%20metodologica%20uso%20del%20tiempo.pdf
[9] Descargando: https://www.funcionpublica.gov.co/eva/gestornormativo/norma.php?i=52105
[10] Descargando

,termino_padre,termino_hijo,definicion_1,fuente_1,definicion_2,fuente_2,definicion_3,fuente_3,terminos_relacionados,origen_link,url_pdf,ruta_pdf_local,estado_descarga,detalle_descarga
0,Actividad Principal,NaN,Actividad que más contribuye al valor agregado...,Sistema de consulta de conceptos estandarizado...,NaN,Clasificación Industrial Internacional Uniform...,NaN,NaN,"Ocupación, empleo, trabajo habitual, Oficio, f...",link_2,https://www.dane.gov.co/files/sen/nomenclatura...,/content/drive/MyDrive/Big Data/PDFs/doc_0000.pdf,ok,6.37 MB
1,Actividad Principal,Estudiante,La educación es un derecho de la persona y un ...,"Constitución política de Colombia, 1991.\nArtí...",NaN,NaN,NaN,NaN,NaN,link_1,https://www.funcionpublica.gov.co/eva/gestorno...,None,no_es_pdf,Content-Type: text/html; charset=utf-8
2,Actividad Principal,Independiente,Persona natural que realiza su actividad econó...,Decreto No. 1601 de 2022.pdf\nTítulo 7. Artícu...,NaN,NaN,NaN,NaN,NaN,link_1,https://www.minsalud.gov.co/Normatividad_Nuevo...,/content/drive/MyDrive/Big Data/PDFs/doc_0002.pdf,ok,0.32 MB
3,Actividad Principal,Informal,Se refiere al tipo de actividad que carece de ...,SNA2008Spanish.pdf\nHoja 612 - Pag 561 / Punto...,NaN,NaN,NaN,NaN,NaN,link_1,https://unstats.un.org/Unsd/nationalaccount/do...,/content/drive/MyDrive/Big Data/PDFs/doc_0003.pdf,ok,5.81 MB
4,Actividad Principal,Incapacidad permanente,Se considera como incapacitado permanente parc...,Ley_776_de_2002.pdf\nPag. 2 - Artículo 5,NaN,NaN,NaN,NaN,NaN,link_1,https://www.funcionpublica.gov.co/eva/gestorno...,None,no_es_pdf,Content-Type: text/html; charset=utf-8


In [ ]:
!pip install pdfminer.six

from pdfminer.high_level import extract_text

def extraer_texto_pdf_local(ruta_pdf):
    """
    Versión simple: solo usa pdfminer, SIN OCR.
    Devuelve: (texto, metodo_extraccion, error_detalle)
    """
    try:
        texto = extract_text(ruta_pdf) or ""
        texto = texto.strip()
        return texto, "pdfminer", None
    except Exception as e:
        return "", "error_pdfminer", str(e)

# ------------------------------------------------------------------
# USO CON df_pdfs
# ------------------------------------------------------------------
df_ok = df_pdfs[df_pdfs["estado_descarga"] == "ok"].copy()
df_error = df_pdfs[df_pdfs["estado_descarga"] != "ok"].copy()

textos = []
metodos = []
errores = []

for i, row in df_ok.iterrows():
    ruta = row["ruta_pdf_local"]
    print(f"[{i}] Extrayendo texto de: {ruta}")
    texto, metodo, error = extraer_texto_pdf_local(ruta)
    textos.append(texto)
    metodos.append(metodo)
    errores.append(error)

df_ok["texto_extraido"] = textos
df_ok["metodo_extraccion"] = metodos
df_ok["error_extraccion"] = errores

df_ok["largo_texto"] = df_ok["texto_extraido"].fillna("").apply(len)

print("Total PDFs con algún texto extraído:", (df_ok["largo_texto"] > 0).sum())
print("Total PDFs sin texto:", (df_ok["largo_texto"] == 0).sum())


[0] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0000.pdf
[2] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0002.pdf
[3] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0003.pdf


[5] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0005.pdf


[6] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0006.pdf


[7] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0007.pdf


[8] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0008.pdf


...

[30] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0030.pdf
[31] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0031.pdf
[32] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0032.pdf
[33] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0033.pdf
[34] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0034.pdf
[36] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0036.pdf
[37] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0037.pdf
[38] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0038.pdf
[39] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0039.pdf
[40] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0040.pdf
[41] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0041.pdf
[42] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0042.pdf
[43] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0043.pdf
[44] Extraye

[79] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0079.pdf
[80] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0080.pdf
[81] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0081.pdf
[82] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0082.pdf
[89] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0089.pdf
[90] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0090.pdf
[91] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0091.pdf
[92] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0092.pdf
[93] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0093.pdf
[94] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0094.pdf
[95] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0095.pdf
[98] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0098.pdf
[99] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0099.pdf
[101] Extray

[128] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0128.pdf
[138] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0138.pdf
[139] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0139.pdf
[141] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0141.pdf
[143] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0143.pdf
[144] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0144.pdf
[146] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0146.pdf
[148] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0148.pdf
[150] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0150.pdf
[154] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0154.pdf
[155] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0155.pdf
[156] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0156.pdf
[157] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0157.pdf

[162] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0162.pdf


[163] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0163.pdf
[164] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0164.pdf
[165] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0165.pdf
[166] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0166.pdf
[167] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0167.pdf
[168] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0168.pdf
[169] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0169.pdf
Total PDFs con algún texto extraído: 131
Total PDFs sin texto: 2


In [ ]:
len(df_pdfs)                          # total filas después del filtrado a 120
df_pdfs["estado_descarga"].value_counts()

,count
estado_descarga,
ok,133
no_es_pdf,26
error_descarga,11


In [ ]:
df_ok_final = df_pdfs[df_ok["estado_descarga"] == "ok"].copy()

==========================
# Lectura de PDFs para indexación
==========================

In [ ]:
import os
import pandas as pd

# 1. Ruta donde YA tienes los PDFs descargados
CARPETA_PDFS = "/content/drive/MyDrive/Big Data/PDFs"

# Listar todos los archivos PDF de la carpeta
archivos = sorted([
    f for f in os.listdir(CARPETA_PDFS)
    if f.lower().endswith(".pdf")
])

print(f"Total PDFs encontrados en la carpeta: {len(archivos)}")

# 2. Crear DataFrame base solo con info de los archivos
df_docs = pd.DataFrame({
    "nombre_archivo": archivos
})

df_docs["ruta_pdf_local"] = df_docs["nombre_archivo"].apply(
    lambda x: os.path.join(CARPETA_PDFS, x)
)

# Crear un id_documento secuencial
df_docs = df_docs.reset_index(drop=True)
df_docs["id_documento"] = df_docs.index + 1

df_docs.head()



Total PDFs encontrados en la carpeta: 133


,nombre_archivo,ruta_pdf_local,id_documento
0,doc_0000.pdf,/content/drive/MyDrive/Big Data/PDFs/doc_0000.pdf,1
1,doc_0002.pdf,/content/drive/MyDrive/Big Data/PDFs/doc_0002.pdf,2
2,doc_0003.pdf,/content/drive/MyDrive/Big Data/PDFs/doc_0003.pdf,3
3,doc_0005.pdf,/content/drive/MyDrive/Big Data/PDFs/doc_0005.pdf,4
4,doc_0006.pdf,/content/drive/MyDrive/Big Data/PDFs/doc_0006.pdf,5


In [ ]:
# 3. Instalar / importar pdfminer
!pip install pdfminer.six -q
from pdfminer.high_level import extract_text

def extraer_texto_pdf_local(ruta_pdf):
    """
    Extrae texto de un PDF local usando pdfminer.
    Devuelve: (texto, metodo_extraccion, error_detalle)
    """
    try:
        texto = extract_text(ruta_pdf) or ""
        texto = texto.strip()
        if len(texto) == 0:
            return "", "pdfminer_sin_texto", None
        else:
            return texto, "pdfminer", None
    except Exception as e:
        return "", "error_pdfminer", str(e)

# 4. Aplicar extracción a todos los PDFs
textos = []
metodos = []
errores = []

for i, row in df_docs.iterrows():
    ruta = row["ruta_pdf_local"]
    print(f"[{i}] Extrayendo texto de: {ruta}")
    texto, metodo, error = extraer_texto_pdf_local(ruta)
    textos.append(texto)
    metodos.append(metodo)
    errores.append(error)

df_docs["texto_extraido"] = textos
df_docs["metodo_extraccion"] = metodos
df_docs["error_extraccion"] = errores

# 5. Largo del texto de cada PDF (número de caracteres)
df_docs["largo_texto"] = df_docs["texto_extraido"].fillna("").apply(len)

print("Total PDFs:", len(df_docs))
print("PDFs con algún texto extraído:", (df_docs["largo_texto"] > 0).sum())
print("PDFs sin texto:", (df_docs["largo_texto"] == 0).sum())

# Para ver los largos exactos por archivo:
df_docs[["id_documento", "nombre_archivo", "largo_texto"]].head(20)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 43.9 MB/s eta 0:00:00
[0] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0000.pdf
[1] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0002.pdf
[2] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0003.pdf


[3] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0005.pdf
[4] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0006.pdf


[5] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0007.pdf


[6] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0008.pdf


[7] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0010.pdf
[8] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0011.pdf
[9] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0012.pdf
[10] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0013.pdf
[11] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0014.pdf


...

[23] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0030.pdf
[24] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0031.pdf
[25] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0032.pdf
[26] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0033.pdf
[27] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0034.pdf
[28] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0036.pdf
[29] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0037.pdf
[30] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0038.pdf
[31] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0039.pdf
[32] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0040.pdf
[33] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0041.pdf
[34] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0042.pdf
[35] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0043.pdf
[36] Extraye

[69] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0079.pdf
[70] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0080.pdf
[71] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0081.pdf
[72] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0082.pdf
[73] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0089.pdf
[74] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0090.pdf
[75] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0091.pdf
[76] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0092.pdf
[77] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0093.pdf
[78] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0094.pdf
[79] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0095.pdf
[80] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0098.pdf
[81] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0099.pdf
[82] Extraye

[109] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0138.pdf
[110] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0139.pdf
[111] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0141.pdf
[112] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0143.pdf
[113] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0144.pdf
[114] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0146.pdf
[115] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0148.pdf
[116] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0150.pdf
[117] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0154.pdf
[118] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0155.pdf
[119] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0156.pdf
[120] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0157.pdf
[121] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0158.pdf

[126] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0163.pdf
[127] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0164.pdf
[128] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0165.pdf
[129] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0166.pdf
[130] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0167.pdf
[131] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0168.pdf
[132] Extrayendo texto de: /content/drive/MyDrive/Big Data/PDFs/doc_0169.pdf
Total PDFs: 133
PDFs con algún texto extraído: 131
PDFs sin texto: 2


,id_documento,nombre_archivo,largo_texto
0,1,doc_0000.pdf,1423365
1,2,doc_0002.pdf,19452
2,3,doc_0003.pdf,4252435
3,4,doc_0005.pdf,6979
4,5,doc_0006.pdf,26101
5,6,doc_0007.pdf,26101
6,7,doc_0008.pdf,26101
7,8,doc_0010.pdf,124912
8,9,doc_0011.pdf,90050
9,10,doc_0012.pdf,122412


In [ ]:
df_docs[["id_documento", "nombre_archivo", "largo_texto"]]

,id_documento,nombre_archivo,largo_texto
0,1,doc_0000.pdf,1423365
1,2,doc_0002.pdf,19452
2,3,doc_0003.pdf,4252435
3,4,doc_0005.pdf,6979
4,5,doc_0006.pdf,26101
...,...,...,...
128,129,doc_0165.pdf,102757
129,130,doc_0166.pdf,6387
130,131,doc_0167.pdf,287679
131,132,doc_0168.pdf,43234


In [ ]:
# Asegúrate de que no haya NaN
df_docs["largo_texto"] = df_docs["largo_texto"].fillna(0).astype(int)

# 1) Más de 1000 caracteres
mas_1000 = (df_docs["largo_texto"] > 1000).sum()

# 2) Entre 1 y 1000 caracteres
menos_1000 = ((df_docs["largo_texto"] > 0) & (df_docs["largo_texto"] <= 1000)).sum()

# 3) Igual a 0 caracteres
ceros = (df_docs["largo_texto"] == 0).sum()

print("PDFs con > 1000 caracteres:", mas_1000)
print("PDFs con 1–1000 caracteres:", menos_1000)
print("PDFs con 0 caracteres:", ceros)

# (opcional) comprobar que suman el total
print("Total filas df_docs:", len(df_docs))
print("Suma de las tres categorías:", mas_1000 + menos_1000 + ceros)


PDFs con > 1000 caracteres: 131
PDFs con 1–1000 caracteres: 0
PDFs con 0 caracteres: 2
Total filas df_docs: 133
Suma de las tres categorías: 133


==========================
# UNIR MATRIZ BASE CON MATRIZ DE EXTRACCIÓN DE TEXTOS
==========================

In [ ]:
## PARA QUE ESTE CODIGO FUNCIONE RECONSTRUIR LA MATRIZ DE LENGUAJE CONTROLADO
import re

# df_docs ya lo tienes con nombre_archivo y largo_texto
df_docs["id_local"] = (
    df_docs["nombre_archivo"]
      .str.extract(r"doc_(\d+)\.pdf")[0]
      .astype(int)
)



In [ ]:
for i, row in df_pdfs.iterrows():
    nombre_archivo = f"doc_{i:04d}.pdf"

In [ ]:
# Después de crear df_pdfs = pd.DataFrame(filas_pdf)
df_pdfs = df_pdfs.reset_index(drop=True)
df_pdfs["id_local"] = df_pdfs.index  # 0,1,2,...


In [ ]:
df_merged = df_docs.merge(
    df_pdfs,
    on="id_local",
    how="left",
    suffixes=("_pdf", "_base")
)

df_merged.head()


,nombre_archivo,ruta_pdf_local,id_documento,texto_extraido,metodo_extraccion,error_extraccion,largo_texto,id_local,termino_padre,termino_hijo,definicion_1,fuente_1,definicion_2,fuente_2,definicion_3,fuente_3,terminos_relacionados,origen_link,url_pdf
0,doc_0000.pdf,/content/drive/MyDrive/Big Data/PDFs/doc_0000.pdf,1,Clasiﬁcación \nIndustrial\nInternacional \nUni...,pdfminer,None,1423365,0,Actividad Principal,NaN,Actividad que más contribuye al valor agregado...,Sistema de consulta de conceptos estandarizado...,NaN,Clasificación Industrial Internacional Uniform...,NaN,NaN,"Ocupación, empleo, trabajo habitual, Oficio, f...",link_2,https://www.dane.gov.co/files/sen/nomenclatura...
1,doc_0002.pdf,/content/drive/MyDrive/Big Data/PDFs/doc_0002.pdf,2,REPÚBLICA DE COLOMBIA \n\nSECRETNRIAJURiDICA \...,pdfminer,None,19452,2,Actividad Principal,Independiente,Persona natural que realiza su actividad econó...,Decreto No. 1601 de 2022.pdf\nTítulo 7. Artícu...,NaN,NaN,NaN,NaN,NaN,link_1,https://www.minsalud.gov.co/Normatividad_Nuevo...
2,doc_0003.pdf,/content/drive/MyDrive/Big Data/PDFs/doc_0003.pdf,3,Sistema de \nCuentas \nNacionales \n2008\n\nC...,pdfminer,None,4252435,3,Actividad Principal,Informal,Se refiere al tipo de actividad que carece de ...,SNA2008Spanish.pdf\nHoja 612 - Pag 561 / Punto...,NaN,NaN,NaN,NaN,NaN,link_1,https://unstats.un.org/Unsd/nationalaccount/do...
3,doc_0005.pdf,/content/drive/MyDrive/Big Data/PDFs/doc_0005.pdf,4,Departamento Administrativo de la Función Públ...,pdfminer,None,6979,5,Actividad Principal,Trabajo de cuidado no remunerado,"Servicios domésticos, personales y de cuidados...",Ley 1413 de 2010\nArtículo 2 - Definición Trab...,Trabajo no remunerado: Comprende las actividad...,Guía Metodológica para las Mediciones UDT en A...,Trabajo Doméstico no remunerado: Incluye una a...,Guía Metodológica para las Mediciones UDT en A...,NaN,link_1,https://www.funcionpublica.gov.co/eva/gestorno...
4,doc_0006.pdf,/content/drive/MyDrive/Big Data/PDFs/doc_0006.pdf,5,GT Género de la CEA-CEPAL\n\nGuía Metodológica...,pdfminer,None,26101,6,Actividad Principal,Trabajo de cuidado no remunerado,"Servicios domésticos, personales y de cuidados...",Ley 1413 de 2010\nArtículo 2 - Definición Trab...,Trabajo no remunerado: Comprende las actividad...,Guía Metodológica para las Mediciones UDT en A...,Trabajo Doméstico no remunerado: Incluye una a...,Guía Metodológica para las Mediciones UDT en A...,NaN,link_2,https://rtc-cea.cepal.org/sites/default/files/...


In [ ]:
## FILTRAR POR LOS PDFS cCON MÍNIMO 1000 CARACTERES
df_con_texto = df_merged[df_merged["largo_texto"] > 0].copy()

# Contar por rango
mas_1000  = (df_con_texto["largo_texto"] >= 1000).sum()
menos_1000 = ((df_con_texto["largo_texto"] > 0) & (df_con_texto["largo_texto"] < 1000)).sum()
ceros      = (df_merged["largo_texto"] == 0).sum()

print(">= 1000 caracteres:", mas_1000)
print("Entre 1 y 999 caracteres:", menos_1000)
print("Sin texto (0 caracteres):", ceros)


>= 1000 caracteres: 131
Entre 1 y 999 caracteres: 0
Sin texto (0 caracteres): 2


In [ ]:
df_buenos = df_con_texto[df_con_texto["largo_texto"] >= 1000].copy()

In [ ]:
import pandas as pd

# Partimos desde df_buenos
df_index = df_buenos.copy().reset_index(drop=True)

print("Total documentos para indexar:", len(df_index))


Total documentos para indexar: 131


In [ ]:
def unir_definiciones(row):
    partes = []
    for col in ["definicion_1", "definicion_2", "definicion_3"]:
        if col in row:
            val = row[col]
            if pd.notna(val):
                txt = str(val).strip()
                if txt:
                    partes.append(txt)
    return " ".join(partes)

df_index["definicion_unificada"] = df_index.apply(unir_definiciones, axis=1)


In [ ]:
cols_texto = [
    "termino_padre", "termino_hijo",
    "definicion_1", "definicion_2", "definicion_3",
    "fuente_1", "fuente_2", "fuente_3",
    "terminos_relacionados",
    "origen_link", "url_pdf",
    "definicion_unificada", "texto_extraido"
]

for c in cols_texto:
    if c in df_index.columns:
        df_index[c] = df_index[c].fillna("")


In [ ]:
df_index_export = df_index.rename(columns={
    "id_documento": "id",                 # ID único del doc
    "nombre_archivo": "file_name",
    "ruta_pdf_local": "file_path",
    "termino_padre": "term_parent",
    "termino_hijo": "term_child",
    "terminos_relacionados": "related_terms",
    "definicion_unificada": "definition",
    "texto_extraido": "pdf_text",
    "largo_texto": "pdf_length",
    "url_pdf": "source_url",
    "origen_link": "source_field",
})


In [ ]:
df_index_export[[
    "id", "file_name", "term_parent", "term_child",
    "definition", "pdf_length"
]].head()


,id,file_name,term_parent,term_child,definition,pdf_length
0,1,doc_0000.pdf,Actividad Principal,,Actividad que más contribuye al valor agregado...,1423365
1,2,doc_0002.pdf,Actividad Principal,Independiente,Persona natural que realiza su actividad econó...,19452
2,3,doc_0003.pdf,Actividad Principal,Informal,Se refiere al tipo de actividad que carece de ...,4252435
3,4,doc_0005.pdf,Actividad Principal,Trabajo de cuidado no remunerado,"Servicios domésticos, personales y de cuidados...",6979
4,5,doc_0006.pdf,Actividad Principal,Trabajo de cuidado no remunerado,"Servicios domésticos, personales y de cuidados...",26101


In [ ]:
ruta_jsonl = "/content/drive/MyDrive/Big Data/docs_lenguaje_controlado.jsonl"

df_index_export.to_json(
    ruta_jsonl,
    orient="records",
    lines=True,
    force_ascii=False
)

print("Archivo JSONL guardado en:", ruta_jsonl)


Archivo JSONL guardado en: /content/drive/MyDrive/Big Data/docs_lenguaje_controlado.jsonl


In [ ]:
ruta_json = "/content/drive/MyDrive/Big Data/docs_lenguaje_controlado.json"

df_index_export.to_json(
    ruta_json,
    orient="records",
    force_ascii=False,
    indent=2  # si tu versión de pandas lo permite
)

print("Archivo JSON guardado en:", ruta_json)


Archivo JSON guardado en: /content/drive/MyDrive/Big Data/docs_lenguaje_controlado.json
